### **Objective:**
To understand and gain insights from a retail dataset by performing various exploratory data analyses, data visualization, and data modelling.


### **By the end of this project:**

* Customers will be grouped in clusters to identify each groups behavoiural pattern

* A product recommendation model will be created for better customer experience.


### **Dataset Columns:**

- **InvoiceNo:** Invoice number. A unique number per invoice.
- **StockCode:** Product code. A unique number per product.
- **Description:** Product description.
- **Quantity:** The number of products sold per invoice.
- **InvoiceDate:** The date and time of the invoice.
- **UnitPrice:** The price of one unit of the product.
- **CustomerID:** Customer identification number.
- **Country:** The country where the customer resides.


## 1. Data Preprocessing and Cleaning:


1.1. Import necessary libraries and read the dataset:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

import warnings
warnings.simplefilter('once')
warnings.filterwarnings('ignore',category=DeprecationWarning)

1.2. Display the top 10 rows of the dataframe:

In [ ]:
data = pd.read_csv(r'C:\Users\chukw\OneDrive - St. Clair College\Data Anaytics for Business_Semester 3\DAB303 - MARKETING ANALYTICS\LABS\project 2\Sales_data.csv', encoding='ISO-8859-1')
data.head(10)

1.3. Check for missing values:



In [ ]:
data.isna().sum()

1.4. Convert the InvoiceDate column to datetime format:



In [ ]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')
data.info()

1.5. Add a new column 'TotalPrice' to the dataframe which is the product of 'UnitPrice' and 'Quantity':



In [ ]:
data['TotalPrice'] = data['UnitPrice'] * data['Quantity']
data.sample(2)

## 2. Exploratory Data Analysis:


2.1. How many unique products are there in the dataset?


In [ ]:
print(f'There are {data['StockCode'].nunique()} unique products in the dataset') # stock code may be better because Description may have 1 items described in multiple ways
# however each StockCode should be unique to a product

2.2. Which are the top 10 products (using StockCode) sold by quantity?



In [ ]:
top_10_products = data.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False).head(10)
top_10_products = pd.DataFrame(top_10_products)
top_10_products

2.3. How many unique customers are there in the dataset?



In [ ]:
unique_customers = data['CustomerID'].nunique()
print(f'There are {unique_customers} unique customers in the dataset')

2.4. Which country has the maximum number of unique customers?



In [ ]:
unique_customers_per_country = int(data.groupby('Country')['CustomerID'].nunique().sort_values(ascending=False).head(1).values[0])

print(f"United Kingdom has the maximum number of unique customers \
with {unique_customers_per_country} customers")

In [ ]:
data['TotalPrice'].max()

2.5. Visualize the distribution of 'TotalPrice' using a histogram.



In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(data['TotalPrice'], bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Total Price')
plt.xlabel('Total Price')
plt.ylabel('Frequency')
plt.figtext(0.1,-0.1,"NOTE: Most of the total prices range from \\$0 - \\$168,469.6. Negative total prices indicates that \
the product was returned and refunded given to the customer",wrap=True,fontsize=10)
plt.show()

## 3. Data Aggregation:


3.1. Compute the total sales (TotalPrice) per country.



In [ ]:
total_sales_per_country = data.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False)
total_sales_per_country = pd.DataFrame(total_sales_per_country)
total_sales_per_country

3.2. Identify the month in which the sales were highest.



In [ ]:
highest_sales_month = data.groupby(data['InvoiceDate'].dt.month)['TotalPrice'].sum().sort_values(ascending=False)
highest_sales_month = pd.DataFrame(highest_sales_month)

highest_month = data.groupby(data['InvoiceDate'].dt.month)['TotalPrice'].sum().sort_values(ascending=False)


print(f'Month {int(highest_month.keys()[0])} (November) has the highest sales of ${int(highest_month.values[0])}')
highest_sales_month

3.3. Compute the average unit price per product.



In [ ]:
average_unit_price_per_product = data.groupby('StockCode')['UnitPrice'].mean().sort_values(ascending=False).reset_index(name='Average price per product')
average_unit_price_per_product = pd.DataFrame(average_unit_price_per_product)
average_unit_price_per_product

3.4. Compute the total quantity sold per customer.



In [ ]:
total_qty_per_customer = data.groupby('CustomerID')['Quantity'].sum().sort_values(ascending=False).reset_index(name='Total Quantity sold')
total_qty_per_customer = pd.DataFrame(total_qty_per_customer)
total_qty_per_customer

## 4. Data Visualization:


4.1. Create a bar chart showcasing the sales (TotalPrice) for each country.



In [ ]:
# Create a bar chart

total_sales_per_country = data.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False)
plt.figure(figsize=(12, 6))
total_sales_per_country.sort_values(ascending=True).plot(kind='barh', color='skyblue', edgecolor='black')
plt.title('Total Sales per Country')
plt.xlabel('Total Sales (TotalPrice)')
plt.ylabel('Country')
plt.yticks()
plt.xticks(rotation=0)
plt.text(1,-11.000005,'NOTE: 1e6 on the x axis representes million \n\nUnited Kingdom has more sales than any other country combined', fontsize=12)
plt.tight_layout()
plt.show()


## Excluding United Kingdom

In [ ]:
excluding_uk = data.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).iloc[1:]


plt.figure(figsize=(12, 6))
excluding_uk.sort_values(ascending=True).plot(kind='barh', color='skyblue', edgecolor='black')
plt.title('Total Sales per Country Excluding United Kingdom')
plt.xlabel('Total Sales (TotalPrice)')
plt.ylabel('Country')
plt.yticks()
plt.xticks(rotation=0)
plt.text(1,-11.000005,'NOTE: By omitting United Kingdom, we see sales from the top countries are not all that far from each other. \nThere may be an opportunity for marketing campaigns in \
those countries to increase sales to potentially same level as the United Kingdom', fontsize=12,wrap=True)
plt.tight_layout()
plt.show()

4.2. Plot a line graph to showcase the trend of sales over time.



In [ ]:
# Group by date and sum TotalPrice
sales_over_time = data.groupby(data['InvoiceDate'].dt.to_period('M'))['TotalPrice'].sum()

# Create a line graph
plt.figure(figsize=(12, 6))
sales_over_time.plot(kind='line', color='blue')
plt.title('Sales Trend Over Time')
plt.xlabel(None)
plt.ylabel('Total Sales (TotalPrice)')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


4.3. Use a scatter plot to visualize the relationship between UnitPrice and Quantity.



In [ ]:
data[data['UnitPrice']<0]

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data, x='UnitPrice', y='Quantity', alpha=0.6, color='purple')
plt.title('Relationship Between Unit Price and Quantity Sold')
plt.xlabel('Unit Price')
plt.ylabel('Quantity')
plt.text(-11500,-150001,'NOTE: Negative values represent returned items, with each negative quantity matched \
by a corresponding positive quantity.',fontsize=12,wrap=True)
plt.tight_layout()
plt.show()


4.4. Plot a heatmap to display the correlation between numeric columns.



Quantity and TotalPrice (0.89): There's a strong positive correlation between these two variables. This makes sense because as the quantity increases, the total price generally increases as well.
UnitPrice and TotalPrice (-0.16): There’s a small negative correlation between UnitPrice and TotalPrice, meaning that an increase in unit price may slightly decrease the total price, though the effect is weak.

In [ ]:
numeric_df = data.select_dtypes(include=['float64','int64'])
numeric_df.drop('CustomerID',axis=1,inplace=True) # dropped customer ID. Although it is numeric, it doesn't add any value to the correlation as it's an identifier
correlation_matrix = numeric_df.corr()

# Create a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', cbar=True)
plt.title('Correlation Heatmap of Numeric Columns')
plt.tight_layout()
plt.text(0.1,3.8,"1) Quantity and TotalPrice (0.89): There's a strong positive correlation between these two variables. This makes sense because as the quantity increases, the total price generally increases as well.\
\n\n 2) UnitPrice and TotalPrice (-0.16): There’s a small negative correlation between UnitPrice and TotalPrice, meaning that an increase in unit price may slightly decrease the total price, though the effect is weak.",wrap=True,fontsize=12)
plt.show()

## 5. Advanced Analysis:


5.1. Segment customers based on their purchase history (Consider factors like frequency of purchases, etc.).



* Segmenting customers based on number of purchases (frequency).

* Frequency above 1000 and below 100 are high and low respectively, anything in between is moderate


In [ ]:
frequency = data.groupby('CustomerID')['InvoiceNo'].count().reset_index(name='invoice_count')


def label_count(count):
    if count >= 100:
        return 'high'
    elif count > 0 and count <= 50:
        return 'low'
    else:
        return 'medium'


data['labels']  = frequency['invoice_count'].apply(label_count)
data['labels'].value_counts().plot(kind='bar',x='labels')
plt.xticks(rotation=0)
plt.title("Customer Segments")
plt.ylabel('Count')
plt.show()

5.3. For the top 5 products (by quantity sold), visualize their monthly sales trend.



In [ ]:
# To get top 5 products by quantity sold
highest_quantity = data.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False).head(5).keys()
highest_quantity = list(highest_quantity)
highest_quantity = data[data['StockCode'].isin(highest_quantity)]

# to get total sales of top 5 products as the months go by
# to get monthly total sales of top 5 products
monthly_sales = highest_quantity.groupby([highest_quantity['InvoiceDate'].dt.to_period('M'),'StockCode'])['Quantity'].sum().unstack()

#plot
monthly_sales.plot(kind='line',figsize=(10,6),marker='o')
plt.ylabel('Quantity Sold')
plt.xlabel('Time Period')
plt.title('Monthly Sales of the Top 5 Produts by Quantity Sold')
plt.legend(title='StockCode')
plt.show()

# Advanced Machine Learning Analysis with Retail Dataset



## 6. Feature Engineering:

6.1 Extract 'Year', 'Month', 'Day', and 'Hour' from the InvoiceDate and create separate columns for each.


In [ ]:
# Extract 'Year', 'Month', 'Day', and 'Hour' from 'InvoiceDate' and create new columns
data['Year'] = data['InvoiceDate'].dt.year
data['Month'] = data['InvoiceDate'].dt.month
data['Day'] = data['InvoiceDate'].dt.day
data['Hour'] = data['InvoiceDate'].dt.hour

# Display the updated DataFrame
data[['InvoiceDate', 'Year', 'Month', 'Day', 'Hour']].head()


6.2 Create a new column `'ReturnFlag'` where if `'Quantity'` is less than zero, it's 1, otherwise 0. This will indicate whether an item was returned.


In [ ]:
def returned_items(items):
    if items < 0:
        return 1
    else:
        return 0 
    
data['ReturnFlag'] = data['Quantity'].apply(returned_items)
# OR  data['ReturnFlag'] = data['Quantity'].apply(lambda x: 1 if x <0 else 0)
data.sample(3)

## 7. Customer Segmentation using Clustering:

7.1. Create a matrix RFM (Recency, Frequency, Monetary) for each customer:
- Recency: Number of days since the last purchase
- Frequency: Number of purchases
- Monetary: Total money spent

In [ ]:
# 1 Segmentation based on frequency
frequency = data.groupby('CustomerID')['InvoiceNo'].count().reset_index(name='invoice_count')

data = data.merge(frequency,on='CustomerID',how='left')
data = data.rename(columns={'invoice_count':'frequency'})



# 2 Segmentation based on monetary value
monetary = data.groupby('CustomerID')['TotalPrice'].sum().reset_index(name='total_spent')

data = data.merge(monetary,how='left',on='CustomerID')
data = data.rename(columns={'total_spent':'monetary'})
data.sample(5)


# 3 segmentation based on recency
last_purchase_date = data['InvoiceDate'].max()
last_purchase_date = pd.to_datetime(last_purchase_date)


most_recent_purchase = data.groupby('CustomerID')['InvoiceDate'].max()
most_recent_purchase = pd.DataFrame(most_recent_purchase)
most_recent_purchase['recency'] = (last_purchase_date - most_recent_purchase['InvoiceDate']).dt.days
most_recent_purchase.drop('InvoiceDate',axis=1,inplace=True)



data = data.merge(most_recent_purchase,on='CustomerID',how='left')

data.sample(3)

7.2. Normalize the RFM matrix with StandardScaler:

In [28]:
# dropping columns with null customerid
data.dropna(subset='CustomerID',inplace=True)

rfm = data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (last_purchase_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalPrice': 'sum'
})

In [ ]:
# Instantiate the StandardScaler
scaler = StandardScaler()

# Fit and transform the RFM matrix
scaled = scaler.fit_transform(rfm)

# Convert the scaled values back into a DataFrame with the original columns
rfm_normalized = pd.DataFrame(scaled,columns=rfm.columns,index=rfm.index)#.reset_index()

# Display the normalized RFM matrix
rfm_normalized.head()

7.3. Use KMeans clustering to segment customers into different groups. Determine the optimal number of clusters using the Elbow method.

In [ ]:
# determining optimal K value

# List to store the sum of squared distances (inertia) for each k
wcss = [] # within-cluster sum of squares

# Testing k values from 1 to 10
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_normalized)
    wcss.append(kmeans.inertia_)

# Plot the Elbow graph to visualize the optimal number of clusters
plt.plot(range(1,11),wcss,marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.text(-1,-4000,'The optimal number is 3 because it is the point before the curve starts to flatten. That means, adding an extra cluster marginally \
reduces the WCSS and very little diminishing returns. \n\nDataset will be split into 3 clusters (groups).',wrap=True)
plt.show()



In [ ]:
# Perform KMeans clustering with the optimal number of clusters
optimal_k = 3
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
rfm_normalized['Cluster'] = kmeans.fit_predict(rfm_normalized)

# Add the cluster labels to the original RFM table
rfm['Cluster'] = rfm_normalized['Cluster']

# Display the RFM table with cluster labels
rfm.sample(3)


In [ ]:
#pip install plotly

In [ ]:
# import plotly.express as px

# fig = px.scatter(rfm, x='InvoiceDate', y='TotalPrice', color='Cluster', hover_data=['InvoiceDate', 'TotalPrice'])
# fig.show()


In [ ]:
# Plot the segments
plt.figure(figsize=(10, 6))
sns.scatterplot(data=rfm, x = 'InvoiceDate', y = 'TotalPrice', hue='Cluster')
#sns.countplot(x='Cluster', data=rfm, order=rfm['Cluster'].value_counts().index)
plt.title('Customer Cluster')
plt.xlabel('Recency (Days)')
plt.ylabel('Total Price')
plt.text(-0.5,-150050,'$\mathbf{Customer\ type\ 2}$ generally are customers that spend the most and have the lowest recency. Theses customers have bought products recently.\n\
$\mathbf{Customer\ type\ 1}$ also have low recency but spend significantly less \n\
$\mathbf{Customer\ type\ 0}$ have the highest recency and spend the least amount',wrap=True)
plt.show()

# Several recommendations can be drawn from this graph.

1. Marketing campaigns can be arranged specifically for customer type 2. This is an attempt to improve their spendimg habits so they eventually upgrade to customer type 2.

2. Offering premium services to customers in the Cluster 2 category could encourage Customer Type 1 to upgrade their purchase habits. This way, Cluster 2 customers can benefit from these services and become more profitable for the business

3. Customer type 2 are the least profitable for the business. They have high recency, meaning they haven't visited in a long time, and their total spend (total price) is almost zero. These customers likely only shop during discount or giveaway periods. They could be targeted specifically during sales seasons.

## 8. Predictive Analytics:

8.1. Can you predict if a customer will return an item? Use the 'ReturnFlag' as the target variable and build a classification model.

- Split data into training and test sets.
- Use features like 'UnitPrice', 'Quantity', etc.
- Evaluate model accuracy, precision, recall, and F1-score.

In [ ]:
# Split the data into features and target
features = data[['Quantity','UnitPrice','TotalPrice','Year','Month','Day','Hour','frequency','monetary','recency']]
target = data['ReturnFlag']

# split the data for training and testing
x_train,x_test,y_train,y_test = train_test_split(features,target,test_size=0.2,random_state= 100)

# using Random Classifier
rfc = RandomForestClassifier(n_estimators=3,random_state=100)

# training the model
rfc.fit(x_train,y_train)

# making prediction based on trained data
prediction = rfc.predict(x_test)

#Evaluation
rfc.score(x_test,y_test)
acc_score = accuracy_score(y_test,prediction)
precision = precision_score(y_test,prediction,average='macro')
recall = recall_score(y_test,prediction,average='macro')
f1 = f1_score(y_test,prediction,average='macro')

print(f'accuaracy score is {acc_score*100}%')
print(f'precision score is {precision*100}%')
print(f'recall is {recall*100}%')
print(f'F1 score is {f1*100}%')

#### The above model was able to predict if a customer will return a product with a high accuracy of 100%

8.2. Predict the `'TotalPrice'` for an invoice using regression models.
- Consider relevant features and handle categorical ones appropriately (e.g., with one-hot encoding).
- Split data, train the model, and evaluate its performance using metrics like MAE, RMSE, and R^2.

In [ ]:
# setting the feature variables
features = data[['Quantity','UnitPrice','Country','Year','Month','Day','Hour','ReturnFlag']]

# One-hot encoding
features = pd.get_dummies(features)

# Values from One-hot encoding returns True and False. Converted to integer to return 0 and 1
features = features.astype('int')

# setting target Variable
target = data['TotalPrice']

# splitting the data
x_train,x_test,y_train,y_test = train_test_split(features,target,test_size=0.2,random_state=100)

# initializing the model
rf = RandomForestRegressor(n_estimators=3,random_state=100)

# training the model
rf.fit(x_train,y_train)

# Using trained model to predict test dataset
prediction = rf.predict(x_test)

mae = mean_absolute_error(y_test,prediction)
rmse = root_mean_squared_error(y_test,prediction)
R_square = rf.score(x_test,y_test)

print(f'Mean absolute error is {mae:.2f}')
print(f'Root mean square error is {rmse:.2f}')
print(f'R square is {R_square:.2f}%')


#### Our model has an R squared of 0.71 which means our model explains 71% of the varaition in Total price, however, 29% remains unexplained

## 9. Association Rule Mining:
- 9.1 Identify frequently bought products together. Use the Apriori algorithm to extract meaningful association rules.
- 9.2 Based on the rules, suggest product bundling strategies to the retail store.

In [37]:
# generally, bundles were identified but their frequency was very low, < 0.0001% of transactions

# i.e minimum support threshold (e.g., 0.01 means items bought together in at least 1% of transactions)


In [38]:
sample = data.sample(1000)
sample.drop(['frequency','monetary','recency'],axis=1,inplace=True)


# using the Apriori Algorithm

# Group by InvoiceNo and StockCode, then pivot to get the binary values for product presence
basket = (sample.groupby(['InvoiceNo', 'StockCode'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# Convert quantities to binary (0 or 1)
basket = basket.map(lambda x: 1 if x > 0 else 0)



In [ ]:
# Applying Apriori
from mlxtend.frequent_patterns import apriori, association_rules

# Apply the Apriori algorithm with a minimum support threshold
# minimum support threshold (e.g., 0.01 means items bought together in at least 1% of transactions)

frequent_itemsets = apriori(basket, min_support=0.000001, use_colnames=True)
print(' An itemset is a group of one or more items. The Apriori algorithm \
identifies itemsets that appear together frequently in a dataset based on a minimum support threshold.\n \
Below are some of the itemsets identified.')
frequent_itemsets.sample(5)
#Frequent Itemsets: An itemset is a group of one or more items. The Apriori algorithm 
# identifies itemsets that appear together frequently in a dataset based on a minimum support threshold.

# these are itemsets, they can be 1 or more. so it's not wrong to see only 1 itemset
#frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x)>1)]



In [ ]:
# Generate association rules
# rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.01)
print('After generating all frequent itemsets, the algorithm derives association rules that meet a minimum confidence threshold. \n\
Meaning - If Stock Code A (antecedents) is purchased, Stock Code B (consequents) is also likely to be purchased. ')
rules.sample(5)


- 9.2 Based on the rules, suggest product bundling strategies to the retail store.

In [ ]:
# Filter the rules with high confidence
high_confidence_rules = rules[rules['confidence'] >= 0.8] # selected minimum confidence of 80%

# Display the filtered rules
print('Stock Code A (antecedents) and Stock Code B (consequents) can be served as bundle packages at the retail stores')
high_confidence_rules.sample(5)

# Advanced EDA Techniques:

10. Pareto Analysis (80/20 Rule):
- Identify the 20% of the products that generate 80% of the revenue.

In [ ]:
# Step 1: Calculate the total revenue for each product
product_revenue = data.groupby('StockCode')['TotalPrice'].sum().reset_index()

# Step 2: Sort products by revenue in descending order
product_revenue = product_revenue.sort_values(by='TotalPrice', ascending=False)

# Step 3: Calculate the cumulative revenue percentage
product_revenue['cum_revenue_perc'] = product_revenue['TotalPrice'].cumsum() / product_revenue['TotalPrice'].sum()

# # Step 4: Identify the top 20% of products that contribute to 80% of the revenue
pareto_products = product_revenue[product_revenue['cum_revenue_perc'] <= 0.8]

# # Output the Pareto products
pareto_products

- Conversely, identify the 20% of the customers responsible for 80% of the sales.    

In [ ]:
# Step 1: Calculate the total sales for each customer
customer_sales = data.groupby('CustomerID')['TotalPrice'].sum().reset_index()
customer_sales

In [ ]:
# Step 1: Calculate the total sales for each customer
customer_sales = data.groupby('CustomerID')['TotalPrice'].sum().reset_index()

# Step 2: Sort customers by sales in descending order
customer_sales = customer_sales.sort_values(by='TotalPrice', ascending=False)

# Step 3: Calculate the cumulative sales percentage
customer_sales['cum_sales_perc'] = customer_sales['TotalPrice'].cumsum() / customer_sales['TotalPrice'].sum()

# Step 4: Identify the top 20% of customers that contribute to 80% of the sales
pareto_customers = customer_sales[customer_sales['cum_sales_perc'] <= 0.8]
pareto_customers

# Advanced Modeling Techniques:

11. Market Basket Analysis Enhancements:
- Dig deeper into association rules. For instance, find rules with a high lift and high confidence.
- Analyze antecedents with more than one item, which can give bundled product suggestions.

In [ ]:
high_lift_and_high_confidence = rules[(rules['confidence']>0.5) & (rules['lift']>190)]
high_lift_and_high_confidence.sample(5)

In [ ]:
bundle_suggestion = high_lift_and_high_confidence[high_lift_and_high_confidence['antecedents'].apply(lambda x: len(x) > 1)]
bundle_suggestion.sample(2)

## 12. Product Recommendation Systems:
- Develop a system to recommend products to users.
- Consider collaborative filtering techniques, matrix factorization, or deep learning-based approaches like neural collaborative filtering.

In [ ]:
data['StockCode'].nunique()

In [ ]:
import pandas as pd

def create_unique_stockcode_df(data):

    # Aggregate the data by StockCode, removing invoiceNumber
    unique_data = data.groupby('StockCode').agg({
        'Description': 'first',
        'Quantity': 'sum',
        'InvoiceDate': 'first',
        'UnitPrice': 'mean',
        'TotalPrice': 'mean',
        'CustomerID': 'first',
        'Country': 'first'
    }).reset_index()

    return unique_data


unique_stockcode_df = create_unique_stockcode_df(data)
unique_stockcode_df = unique_stockcode_df[unique_stockcode_df['TotalPrice']>0]
unique_stockcode_df = unique_stockcode_df.rename(columns={'TotalPrice':'Average Price'})
unique_stockcode_df['Average Price'] = unique_stockcode_df['Average Price'].apply(lambda x: f'${x:.2f}')
unique_stockcode_df.sample(5)


In [97]:
# Product recommendation system : a content based filtering based on similarities in product descriptions

# sample = data.sample(frac=0.4, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the product descriptions
tfidf_matrix = tfidf.fit_transform(unique_stockcode_df['Description'])


In [ ]:
from sklearn.neighbors import NearestNeighbors

# Initialize the Nearest Neighbors model
nn = NearestNeighbors(metric='cosine', algorithm='brute')

# Fit the model
nn.fit(tfidf_matrix)

In [99]:
def get_recommendations_nn(k=10):
    try:
        stock_code = input('Please enter stock code here: ')
        # Check if the stock code exists in the dataset
        if not (unique_stockcode_df['StockCode'] == stock_code).any():
            raise ValueError(f"StockCode {stock_code} not found in the dataset.")

        # Get the index of the product that matches the stock code
        idx = unique_stockcode_df[unique_stockcode_df['StockCode'] == stock_code].index

        # Ensure the index is within the range of the dataset
        if idx >= len(unique_stockcode_df):
            raise IndexError(f"Index {idx} is out of range for the dataset.")

        # Get the TF-IDF vector for the product
        query_vector = tfidf_matrix[idx]

        # Find the k nearest neighbors
        distances, indices = nn.kneighbors(query_vector, n_neighbors=k+1)

        # Get the indices of the similar products
        similar_indices = indices.flatten()[1:]

        # Ensure the indices are within the range of the dataset
        similar_indices = [i for i in similar_indices if i < len(unique_stockcode_df)]
        #
        print(f'Customers that purchase {stock_code} are also interested in some of these products')

        # Return the top k most similar products
        return unique_stockcode_df.iloc[similar_indices][['StockCode', 'Description', 'Average Price']]

    except IndexError as e:
        print(f"IndexError: {e}")
        return pd.DataFrame(columns=['StockCode', 'Description', 'Average Price'])
    except ValueError as e:
        print(f"ValueError: {e}")
        return pd.DataFrame(columns=['StockCode', 'Description', 'Average Price'])


In [ ]:
# Running this function will request a stock code of interest is inserted
get_recommendations_nn()
# INSERT 22544